<a href="https://colab.research.google.com/github/nagamachia/Signate/blob/master/AIQuestAssesment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !ls 'drive/My Drive/Signate/AIQuestAssessment'

In [ ]:
# !nvidia-smi

In [ ]:
# !pip3 uninstall --yes lightgbm && pip3 install --install-option=--gpu lightgbm


In [ ]:
!pip3 install optuna

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import math
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
# import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from optuna.integration import lightgbm as lgb
# from sklearn.pipeline import make_pipeline
# from sklearn.linear_model import (
#     LinearRegression,
#     Ridge,
#     Lasso
# )
%matplotlib inline

In [ ]:
dir = "drive/My Drive/Signate/AIQuestAssessment/"
train = pd.read_csv(dir+"train.csv")
test = pd.read_csv(dir+"test.csv")
test['y'] = 9999999999
alldata = pd.concat([train,test],axis=0).reset_index(drop=True)

In [ ]:
train.head()

,id,accommodates,amenities,bathrooms,bed_type,bedrooms,beds,cancellation_policy,city,cleaning_fee,description,first_review,host_has_profile_pic,host_identity_verified,host_response_rate,host_since,instant_bookable,last_review,latitude,longitude,name,neighbourhood,number_of_reviews,property_type,review_scores_rating,room_type,thumbnail_url,zipcode,y
0,0,6,"{TV,""Wireless Internet"",Kitchen,""Free parking ...",2.0,Real Bed,1.0,4.0,flexible,LA,t,My place is meant for family and a few friends...,2016-07-27,t,f,NaN,2016-07-13,f,2016-07-27,33.788931,-118.154761,The Penthouse,NaN,1,Apartment,60.0,Private room,NaN,90804,138.0
1,1,2,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",1.0,Real Bed,1.0,1.0,strict,DC,t,This is a new listing for a lovely guest bedro...,2016-09-12,t,t,100%,2015-12-30,f,2017-03-31,38.934810,-76.978190,Guest Bedroom in Brookland,Brookland,9,House,100.0,Private room,https://a0.muscache.com/im/pictures/e4d8b51f-6...,20018,42.0
2,2,2,"{TV,Internet,""Wireless Internet"",Kitchen,""Indo...",2.0,Real Bed,1.0,1.0,strict,NYC,t,We're looking forward to your stay at our apt....,2016-06-15,t,f,100%,2016-05-21,t,2017-08-13,40.695118,-73.926240,Clean Modern Room in Lux Apt 1 Block From J Train,Bushwick,27,Apartment,83.0,Private room,https://a0.muscache.com/im/pictures/5ffecc9b-d...,NaN,65.0
3,3,2,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",1.0,Real Bed,1.0,1.0,strict,SF,t,BEST CITY VIEWS - - ROOF DECK W/ BBQ & WiFi - ...,2014-03-15,t,t,100%,2012-06-19,t,2017-09-03,37.796728,-122.411906,BEST views + reviews! 5/5 stars*****,Nob Hill,38,Apartment,95.0,Private room,NaN,94133,166.0
4,4,2,"{TV,Internet,""Wireless Internet"",""Air conditio...",1.0,Real Bed,1.0,1.0,strict,NYC,t,Charming Apartment on the upper west side of M...,2015-08-05,t,t,100%,2015-03-25,f,2017-09-10,40.785050,-73.974691,Charming 1-bedroom - UWS Manhattan,Upper West Side,5,Apartment,100.0,Entire home/apt,https://a0.muscache.com/im/pictures/92879730/5...,10024,165.0


In [ ]:
print('dataframeの行数・列数の確認==>\n', train.shape)
print('indexの確認==>\n', train.index)
print('columnの確認==>\n', train.columns)
print('dataframeの各列のデータ型を確認==>\n', train.dtypes)

dataframeの行数・列数の確認==>
 (55583, 29)
indexの確認==>
 RangeIndex(start=0, stop=55583, step=1)
columnの確認==>
 Index(['id', 'accommodates', 'amenities', 'bathrooms', 'bed_type', 'bedrooms',
       'beds', 'cancellation_policy', 'city', 'cleaning_fee', 'description',
       'first_review', 'host_has_profile_pic', 'host_identity_verified',
       'host_response_rate', 'host_since', 'instant_bookable', 'last_review',
       'latitude', 'longitude', 'name', 'neighbourhood', 'number_of_reviews',
       'property_type', 'review_scores_rating', 'room_type', 'thumbnail_url',
       'zipcode', 'y'],
      dtype='object')
dataframeの各列のデータ型を確認==>
 id                          int64
accommodates                int64
amenities                  object
bathrooms                 float64
bed_type                   object
bedrooms                  float64
beds                      float64
cancellation_policy        object
city                       object
cleaning_fee               object
description             

In [ ]:
train.isnull().sum()[train.isnull().sum()>0].sort_values(ascending=False)

host_response_rate        13704
review_scores_rating      12556
first_review              11908
last_review               11880
thumbnail_url              6145
neighbourhood              5160
zipcode                     716
host_since                  148
host_identity_verified      148
host_has_profile_pic        148
bathrooms                   147
beds                         96
bedrooms                     71
dtype: int64

In [ ]:
test.isnull().sum()[test.isnull().sum()>0].sort_values(ascending=False)

host_response_rate        4595
review_scores_rating      4166
first_review              3956
last_review               3947
thumbnail_url             2071
neighbourhood             1712
zipcode                    250
bathrooms                   53
host_since                  40
host_identity_verified      40
host_has_profile_pic        40
beds                        35
bedrooms                    20
dtype: int64

In [ ]:
# 欠損を含むカラムのデータ型を確認
na_col_list = alldata.isnull().sum()[alldata.isnull().sum()>0].index.tolist() # 欠損を含むカラムをリスト化
alldata[na_col_list].dtypes.sort_values() #データ型

bathrooms                 float64
bedrooms                  float64
beds                      float64
review_scores_rating      float64
first_review               object
host_has_profile_pic       object
host_identity_verified     object
host_response_rate         object
host_since                 object
last_review                object
neighbourhood              object
thumbnail_url              object
zipcode                    object
dtype: object

In [ ]:
# データ型に応じて欠損値を補完する
# floatの場合は0
# objectの場合は'NA'
na_float_cols = alldata[na_col_list].dtypes[alldata[na_col_list].dtypes=='float64'].index.tolist() #float64
na_obj_cols = alldata[na_col_list].dtypes[alldata[na_col_list].dtypes=='object'].index.tolist() #object
print(na_float_cols)
print(na_obj_cols)

['bathrooms', 'bedrooms', 'beds', 'review_scores_rating']
['first_review', 'host_has_profile_pic', 'host_identity_verified', 'host_response_rate', 'host_since', 'last_review', 'neighbourhood', 'thumbnail_url', 'zipcode']


In [ ]:
test['property_type'].value_counts()


Apartment             12177
House                  4185
Condominium             675
Townhouse               443
Loft                    330
Other                   159
Bed & Breakfast         113
Guesthouse              106
Bungalow                 85
Villa                    48
Dorm                     30
Guest suite              27
Cabin                    25
Camper/RV                23
Timeshare                22
Boutique hotel           19
Hostel                   19
Boat                     12
In-law                   11
Serviced apartment        5
Tent                      4
Castle                    4
Chalet                    2
Cave                      1
Lighthouse                1
Hut                       1
Yurt                      1
Name: property_type, dtype: int64

In [ ]:
# float64型で欠損している場合は0を代入
for na_float_col in na_float_cols:
    train.loc[train[na_float_col].isnull(),na_float_col] = 0.0
    test.loc[test[na_float_col].isnull(),na_float_col] = 0.0
# object型で欠損している場合は'NA'を代入
for na_obj_col in na_obj_cols:
    train.loc[train[na_obj_col].isnull(),na_obj_col] = 'NA'
    test.loc[test[na_obj_col].isnull(),na_obj_col] = 'NA'

In [ ]:
# # カテゴリカル変数の特徴量をリスト化
# cat_cols = alldata.dtypes[alldata.dtypes=='object'].index.tolist()
# # 数値変数の特徴量をリスト化
# num_cols = alldata.dtypes[alldata.dtypes!='object'].index.tolist()

In [ ]:
# # カテゴリカル変数をダミー化
# alldata_cat = pd.get_dummies(alldata[cat_cols])
# # データ統合
# all_data = pd.concat([alldata[num_cols],alldata_cat],axis=1)

In [ ]:
train_x = train.drop(['y'], axis=1)
train_y = train['y']
test_x = test.drop(['y'], axis=1)
train_x = train_x.drop(['id','amenities','description','name','first_review','host_since','last_review','thumbnail_url','host_response_rate','neighbourhood','property_type','zipcode'],axis=1)
test_x = test_x.drop(['id','amenities','description','name','first_review','host_since','last_review','thumbnail_url','host_response_rate','neighbourhood','property_type','zipcode'],axis=1)

In [ ]:
# カテゴリカル変数の特徴量をリスト化
cat_cols = train_x.dtypes[train_x.dtypes=='object'].index.tolist()
# 数値変数の特徴量をリスト化
num_cols = train_x.dtypes[train_x.dtypes!='object'].index.tolist()

In [ ]:
from sklearn.preprocessing import LabelEncoder

for c in cat_cols:
  le = LabelEncoder()
  le.fit(train_x[c])
  train_x[c] = le.transform(train_x[c])
  test_x[c] = le.transform(test_x[c])

LabelEncoder()

LabelEncoder()

LabelEncoder()

LabelEncoder()

LabelEncoder()

LabelEncoder()

LabelEncoder()

LabelEncoder()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.2, random_state=0)

In [ ]:
lgb_train = lgb.Dataset(X_train, label=y_train)
lgb_eval = lgb.Dataset(X_test, label=y_test)

params = {
        'objective': 'regression',
        'metric': 'rmse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
    }
categorical_features = cat_cols

model = lgb.train(params,lgb_train,categorical_feature=categorical_features,valid_sets=lgb_eval,early_stopping_rounds=100)

print(model.params)
print(model.best_iteration)
print(model.best_score)

va_pred = model.predict(X_test)
score = mean_squared_error(va_pred,y_test)
print(math.sqrt(score))





  0%|          | 0/7 [00:00<?, ?it/s]



feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1209: UserWarning:

categorical_feature in Dataset is overridden.
New categorical_feature is ['bed_type', 'cancellation_policy', 'city', 'cleaning_fee', 'host_has_profile_pic', 'host_identity_verified', 'instant_bookable', 'room_type']

/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:762: UserWarning:

categorical_feature in param dict is overridden.



[1]	valid_0's rmse: 160.448
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's rmse: 155.003
[3]	valid_0's rmse: 149.085
[4]	valid_0's rmse: 145.005
[5]	valid_0's rmse: 140.485
[6]	valid_0's rmse: 136.853
[7]	valid_0's rmse: 133.808
[8]	valid_0's rmse: 130.906
[9]	valid_0's rmse: 128.56
[10]	valid_0's rmse: 126.494
[11]	valid_0's rmse: 124.684
[12]	valid_0's rmse: 123.347
[13]	valid_0's rmse: 122.064
[14]	valid_0's rmse: 120.859
[15]	valid_0's rmse: 120.046
[16]	valid_0's rmse: 119.166
[17]	valid_0's rmse: 118.476
[18]	valid_0's rmse: 117.773
[19]	valid_0's rmse: 117.284
[20]	valid_0's rmse: 116.905
[21]	valid_0's rmse: 116.426
[22]	valid_0's rmse: 115.874
[23]	valid_0's rmse: 115.529
[24]	valid_0's rmse: 115.22
[25]	valid_0's rmse: 114.94
[26]	valid_0's rmse: 114.773
[27]	valid_0's rmse: 114.419
[28]	valid_0's rmse: 114.231
[29]	valid_0's rmse: 114.039
[30]	valid_0's rmse: 113.916
[31]	valid_0's rmse: 113.683
[32]	valid_0's rmse: 113.493
[33]	valid_0's rmse: 





feature_fraction, val_score: 110.029047:   0%|          | 0/7 [00:03<?, ?it/s]



feature_fraction, val_score: 110.029047:  14%|#4        | 1/7 [00:03<00:20,  3.43s/it][I 2020-08-30 12:47:10,594] Trial 0 finished with value: 110.02904674708775 and parameters: {'feature_fraction': 0.8}. Best is trial 0 with value: 110.02904674708775.




feature_fraction, val_score: 110.029047:  14%|#4        | 1/7 [00:03<00:20,  3.43s/it]

[1]	valid_0's rmse: 160.404
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's rmse: 153.55
[3]	valid_0's rmse: 148.022
[4]	valid_0's rmse: 143.398
[5]	valid_0's rmse: 139.149
[6]	valid_0's rmse: 135.619
[7]	valid_0's rmse: 132.316
[8]	valid_0's rmse: 129.734
[9]	valid_0's rmse: 127.516
[10]	valid_0's rmse: 125.654
[11]	valid_0's rmse: 124.028
[12]	valid_0's rmse: 122.707
[13]	valid_0's rmse: 121.483
[14]	valid_0's rmse: 120.339
[15]	valid_0's rmse: 119.421
[16]	valid_0's rmse: 118.597
[17]	valid_0's rmse: 117.976
[18]	valid_0's rmse: 117.34
[19]	valid_0's rmse: 116.836
[20]	valid_0's rmse: 116.477
[21]	valid_0's rmse: 116.097
[22]	valid_0's rmse: 115.684
[23]	valid_0's rmse: 115.437
[24]	valid_0's rmse: 115.181
[25]	valid_0's rmse: 115.038
[26]	valid_0's rmse: 114.775
[27]	valid_0's rmse: 114.476
[28]	valid_0's rmse: 114.096
[29]	valid_0's rmse: 113.886
[30]	valid_0's rmse: 113.841
[31]	valid_0's rmse: 113.615
[32]	valid_0's rmse: 113.462
[33]	valid_0's rmse:





feature_fraction, val_score: 110.029047:  14%|#4        | 1/7 [00:06<00:20,  3.43s/it]



feature_fraction, val_score: 110.029047:  29%|##8       | 2/7 [00:06<00:16,  3.28s/it][I 2020-08-30 12:47:13,509] Trial 1 finished with value: 110.49869463951956 and parameters: {'feature_fraction': 0.8999999999999999}. Best is trial 0 with value: 110.02904674708775.




feature_fraction, val_score: 110.029047:  29%|##8       | 2/7 [00:06<00:16,  3.28s/it]

...

[1]	valid_0's rmse: 161.628
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's rmse: 157.143
[3]	valid_0's rmse: 151.112
[4]	valid_0's rmse: 146.986
[5]	valid_0's rmse: 143.147
[6]	valid_0's rmse: 139.451
[7]	valid_0's rmse: 137.262
[8]	valid_0's rmse: 133.616
[9]	valid_0's rmse: 131.57
[10]	valid_0's rmse: 128.909
[11]	valid_0's rmse: 126.935
[12]	valid_0's rmse: 125.406
[13]	valid_0's rmse: 123.93
[14]	valid_0's rmse: 122.614
[15]	valid_0's rmse: 121.515
[16]	valid_0's rmse: 120.295
[17]	valid_0's rmse: 119.192
[18]	valid_0's rmse: 118.282
[19]	valid_0's rmse: 117.731
[20]	valid_0's rmse: 117.171
[21]	valid_0's rmse: 116.537
[22]	valid_0's rmse: 115.943
[23]	valid_0's rmse: 115.511
[24]	valid_0's rmse: 115.136
[25]	valid_0's rmse: 114.864
[26]	valid_0's rmse: 114.598
[27]	valid_0's rmse: 114.304
[28]	valid_0's rmse: 114.038
[29]	valid_0's rmse: 113.837
[30]	valid_0's rmse: 113.649
[31]	valid_0's rmse: 113.3
[32]	valid_0's rmse: 113.163
[33]	valid_0's rmse: 1





min_data_in_leaf, val_score: 108.907922:  60%|######    | 3/5 [00:13<00:06,  3.25s/it]



min_data_in_leaf, val_score: 108.907922:  80%|########  | 4/5 [00:13<00:03,  3.24s/it][I 2020-08-30 12:50:42,666] Trial 66 finished with value: 109.16816261699267 and parameters: {'min_child_samples': 10}. Best is trial 66 with value: 109.16816261699267.




min_data_in_leaf, val_score: 108.907922:  80%|########  | 4/5 [00:13<00:03,  3.24s/it]

[1]	valid_0's rmse: 161.628
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's rmse: 157.143
[3]	valid_0's rmse: 151.052
[4]	valid_0's rmse: 146.926
[5]	valid_0's rmse: 143.086
[6]	valid_0's rmse: 139.392
[7]	valid_0's rmse: 137.205
[8]	valid_0's rmse: 133.533
[9]	valid_0's rmse: 131.468
[10]	valid_0's rmse: 128.809
[11]	valid_0's rmse: 126.829
[12]	valid_0's rmse: 125.295
[13]	valid_0's rmse: 123.851
[14]	valid_0's rmse: 122.517
[15]	valid_0's rmse: 121.426
[16]	valid_0's rmse: 120.249
[17]	valid_0's rmse: 119.199
[18]	valid_0's rmse: 118.275
[19]	valid_0's rmse: 117.721
[20]	valid_0's rmse: 117.18
[21]	valid_0's rmse: 116.515
[22]	valid_0's rmse: 115.929
[23]	valid_0's rmse: 115.496
[24]	valid_0's rmse: 115.064
[25]	valid_0's rmse: 114.787
[26]	valid_0's rmse: 114.547
[27]	valid_0's rmse: 114.202
[28]	valid_0's rmse: 114.047
[29]	valid_0's rmse: 113.811
[30]	valid_0's rmse: 113.594
[31]	valid_0's rmse: 113.276
[32]	valid_0's rmse: 113.16
[33]	valid_0's rmse:





min_data_in_leaf, val_score: 108.907922:  80%|########  | 4/5 [00:16<00:03,  3.24s/it]



min_data_in_leaf, val_score: 108.907922: 100%|##########| 5/5 [00:16<00:00,  3.27s/it]


{'objective': 'regression', 'metric': 'rmse', 'verbosity': -1, 'boosting_type': 'gbdt', 'lambda_l1': 0.6185035757493442, 'lambda_l2': 1.0138731660570927e-08, 'num_leaves': 29, 'feature_fraction': 0.5, 'bagging_fraction': 1.0, 'bagging_freq': 0, 'min_child_samples': 20}
237
defaultdict(<class 'dict'>, {'valid_0': {'rmse': 108.90792154183909}})
108.90792154183909


In [ ]:
pred = model.predict(test_x)
pred

array([264.02025808, 132.98796752, 121.39745231, ..., 227.95501371,
       140.33347402,  50.43562267])

In [ ]:
df_pred = pd.Series(pred)
df_pred

0        264.020258
1        132.987968
2        121.397452
3        157.401075
4        123.920604
            ...    
18523    222.728595
18524    108.748561
18525    227.955014
18526    140.333474
18527     50.435623
Length: 18528, dtype: float64

In [ ]:
df_pred.to_csv(dir+'submit.csv',header=False)